<a href="https://colab.research.google.com/github/MD-Ryhan/Bert-Hyperparameter-Tuninug-with-Optuna/blob/main/MuRIL_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.28.0

!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import optuna
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, BertForSequenceClassification, AutoConfig, BertTokenizer, AutoModelForSequenceClassification ,TrainingArguments, Trainer


In [ ]:
train = pd.read_excel("offtrain.xlsx")
validation = pd.read_excel("offval.xlsx")
test = pd.read_excel("offtest.xlsx")

In [ ]:
from datasets import Dataset, DatasetDict
import datasets

train_df = pd.DataFrame({
     "text" : train["text"],
     "labels" : train['label']

})
val_df = pd.DataFrame({
     "text" : validation["text"],
     "labels" : validation['label']

})

test_df = pd.DataFrame({
     "text" : test["text"],
     "labels" : test['label']

})

train_dataset = Dataset.from_dict(train_df)
val_dataset = Dataset.from_dict(val_df)
test_dataset = Dataset.from_dict(test_df)
dataset = datasets.DatasetDict({"train":train_dataset,"validation":val_dataset,"test":test_dataset})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Hate-speech-CNERG/urdu-abusive-MuRIL')

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
#dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/5475 [00:00<?, ? examples/s]

Map:   0%|          | 0/609 [00:00<?, ? examples/s]

Map:   0%|          | 0/1521 [00:00<?, ? examples/s]

In [ ]:
# Define the hyperparameter search space
import transformers
SAVE_DIR = 'opt-test'
def objective(trial):
    # Define hyperparameters to search for
    training_args = TrainingArguments(
    output_dir=SAVE_DIR,
    learning_rate = trial.suggest_float("learning_rate", 0.99e-5, 2e-5,step=0.2525),
    weight_decay = trial.suggest_float("weight_decay", 0.001, 0.1,step=0.0198),
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    num_train_epochs = trial.suggest_int('num_train_epochs', 3, 10),

    )
    #Set the default values for attention dropout, hidden dropout, and warmup
    attention_dropout = trial.suggest_float("attention_dropout", 0.03, 0.1, step=0.2425)
    hidden_dropout = trial.suggest_float("hidden_dropout", 0.03, 0.1, step=0.2425)
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.04, 0.1, step=0.02)


    # Load the XLM-RoBERTa tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('Hate-speech-CNERG/urdu-abusive-MuRIL')
    model = AutoModelForSequenceClassification.from_pretrained('Hate-speech-CNERG/urdu-abusive-MuRIL')

    # Set the dropout rates and warmup ratio in the model configuration
    model.config.attention_probs_dropout_prob = attention_dropout
    model.config.hidden_dropout_prob = hidden_dropout
    model.config.attention_probs_dropout_prob = attention_dropout
    model.config.hidden_dropout_prob = hidden_dropout
    model.config.warmup_ratio = warmup_ratio

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test']
        )

    trainer.train()
    results = trainer.evaluate()
    return results

In [ ]:
# Load the training, validation, and testing data
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [ ]:
from transformers.optimization_tf import WarmUp

In [ ]:
# Create the Optuna study and start the hyperparameter search
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

In [ ]:
# Print the best trial and its validation accuracy
print('Best trial:')
trial = study.best_trial
print('  Value: {:.6f}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))



In [ ]:
# Train the final model on the combined training and validation data using the best hyperparameters
best_learning_rate = trial.params["learning_rate"]
best_num_hidden_layers = trial.params["num_hidden_layers"]
best_num_attention_heads = trial.params["num_attention_heads"]
best_dropout_rate = trial.params["dropout_rate"]
best_batch_size = trial.params["batch_size"]
best_warmup_ratio = trial.params["warmup_ratio"]
best_weight_decay = trial.params["weight_decay"]
best_num_epochs = trial.params["num_train_epochs"]